In [5]:
import openpyxl
import glob
import pandas as pd
import re

def from_sheets(file):
    
    sheet_names = pd.ExcelFile(file).sheet_names[1:]
    
    one_from_file= pd.DataFrame(pd.read_excel(file, sheet_name=sheet_names[0], header=0, convert_float=True).drop(0,axis=0))
    one_from_file['Käyttöönottopvä'] = one_from_file['Käyttöönottopvä'].astype('datetime64[ns]')
    one_from_file['Päätöspäivä'] = one_from_file['Päätöspäivä'].astype('datetime64[ns]')
    
    
    for sheet in sheet_names[1:]:
    
        data = pd.read_excel(file, sheet_name=sheet, header=0, convert_float=True).drop(0,axis=0)
        data['Käyttöönottopvä'] = data['Käyttöönottopvä'].astype('datetime64[ns]')
        data['Päätöspäivä'] = data['Päätöspäivä'].astype('datetime64[ns]')
        one_from_file = one_from_file.append(data)
        
    return one_from_file




#for year in car_files2:
def all_data_from_files():
    
    car_files = []
    for file in glob.glob("*.xls"):
        car_files.append(file)
    car_files
    

    
    data_1st_year = from_sheets(car_files[0])
    
    for year in car_files[1:]:
        
        
        data = from_sheets(year)
        
        data_1st_year = data_1st_year.append(data)
        
        
    data_1st_year["Autovero"] = pd.to_numeric(data_1st_year["Autovero"], errors='coerce')
    data_1st_year["Ajokm/1000"] = pd.to_numeric(data_1st_year["Ajokm/1000"], errors='coerce')
    data_1st_year["Verotusarvo"] = pd.to_numeric(data_1st_year["Verotusarvo"], errors='coerce')    
    
    
    return data_1st_year
    
def engine_power(col):
    '''
    Extracts engine power from string
    '''
    #x = re.findall('\d+[KW]', str(col)) # Returns also 4WD
    x = re.findall(' \d+[K]', str(col)) # This works, matches numbers followed by K
    x = re.sub('[^0-9]', '', str(x))
    return(x)

def engine_size(col):
    '''
    Extracts engine size from string
    '''
    x = re.findall('\d\.\d', str(col)) # This works perfectly
    x = re.sub('[^0-9.]','', str(x))
    return(x)

def door_number(col):
    '''
    Extracts number of doors from string
    '''
    x = re.findall(' \d+[D]', str(col))
    x = re.sub('[^0-9.]','', str(x))
    return(x)

def extract_D_one(col):
    
    x = re.findall('\d\.\d[A-Z]+', str(col))
    x = re.sub('[0-9.]',"", str(x)).strip("[']")
    return(x)

    
def data_munge(data):
    '''
    Column [Mallin tarkennin] has information about a car 
    
    This function extracts information from column, and creates new columns based on info
    '''
    # Clean data 
    #data['Mallin tarkennin'] = data['Mallin tarkennin'].apply(lambda x: x.replace(",", "."))
    data['Mallin tarkennin'] = data['Mallin tarkennin'].str.replace(",",".")

    data['Mallin tarkennin'] = data['Mallin tarkennin'].str.upper() # All letters to capital
    data['Mallin tarkennin'] = data['Mallin tarkennin'].map(lambda x: re.sub(' KW', 'KW', str(x))) # Remove whitespace before KW
    
    # New columns
    data['diesel'] = data['Mallin tarkennin'].str.count(pat=' D ') # Look for D, if found prints 1
    data['aut'] = data['Mallin tarkennin'].str.count(pat='AUT') # Look for AUT, if found print 1
    data['neliveto'] = data['Mallin tarkennin'].str.count(pat='4WD') # Look for 4WD, if found print 1
    
    data['year_month'] = data["Päätöspäivä"].dt.strftime('%Y-%m')
    data['year'] = data['Päätöspäivä'].dt.year # Year column
    data['month'] = data['Päätöspäivä'].dt.month # Month column
    data['car_age'] = (data['Päätöspäivä']-(data['Käyttöönottopvä']))/ pd.Timedelta(365, unit='d') # Car age
    data['engine_powers'] = data['Mallin tarkennin'].apply(engine_power) # Engine power in KW:s
    data['engine_size'] = data['Mallin tarkennin'].apply(engine_size) # Engine size in cm
    data['doors'] = data['Mallin tarkennin'].apply(door_number)
    data['extract_D_one'] = data['Mallin tarkennin'].apply(extract_D_one)
    
    # Correct data types
    data['engine_powers'] = pd.to_numeric(data['engine_powers'])
    data['engine_size'] = pd.to_numeric(data['engine_size'])
    data['doors'] = pd.to_numeric(data['doors'])

    
    return(data)



In [6]:
data = all_data_from_files()

C:\Users\Manu\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [7]:
data = data_munge(data)

In [8]:
data.to_excel(r'data_with_features.xlsx', index = False)

In [27]:
data.dtypes

Ajokm/1000                                float64
Autovero                                  float64
Kunto A=Alennettu                          object
Kunto H=Huono N=Normaali Y=Hyvä            object
Käyttöönottopvä                    datetime64[ns]
Malli                                      object
Mallin tarkennin                           object
Merkki                                     object
Päätöspäivä                        datetime64[ns]
Verotusarvo                               float64
dtype: object

In [35]:
data.head()

,Ajokm/1000,Autovero,Kunto A=Alennettu,Kunto H=Huono N=Normaali Y=Hyvä,Käyttöönottopvä,Malli,Mallin tarkennin,Merkki,Päätöspäivä,Verotusarvo,diesel,aut,neliveto,year_month,year,month,car_age,engine_powers,engine_size,doors
1,185.0,1319.08,NaN,N,2004-11-10,156,1.9 D 5D STW 110KW,ALFA ROMEO,2012-01-11,5384.00,1,0,0,2012-01,2012,1,7.172603,110.0,1.9,5.0
2,78.0,5229.94,NaN,N,2008-06-11,159,2.4 D 5D STW 4WD 154KW,ALFA ROMEO,2012-01-20,18350.70,1,0,1,2012-01,2012,1,3.610959,154.0,2.4,5.0
3,101.0,7064.43,NaN,N,2007-11-12,159,3.2 5D STW 4WD 191KW,ALFA ROMEO,2012-01-26,16352.85,0,0,1,2012-01,2012,1,4.208219,191.0,3.2,5.0
4,241.0,704.45,NaN,N,2000-05-30,166,2.5 4D 140KW,ALFA ROMEO,2012-01-03,2243.50,0,0,0,2012-01,2012,1,11.602740,140.0,2.5,4.0
5,149.0,1211.13,NaN,N,1999-04-09,166,3.0 4D AUT 166KW,ALFA ROMEO,2012-01-23,3773.00,0,1,0,2012-01,2012,1,12.800000,166.0,3.0,4.0
